In [ ]:
!pip uninstall transformers -y
!pip install transformers[torch] datasets

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu1

In [ ]:
# prompt: google mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# dataset load

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch

# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the CSV files
train_df = pd.read_csv('/content/drive/MyDrive/data/train_dataset.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/data/valid_dataset.csv')

# Convert DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# Load tokenizer and model
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples['hs'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/145949 [00:00<?, ? examples/s]

Map:   0%|          | 0/17965 [00:00<?, ? examples/s]

# model architecture define

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# 모델과 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModel.from_pretrained("klue/bert-base", output_hidden_states=True, output_attentions=True)

# 입력 데이터 설정
text = "여기에 분석할 문장을 입력하세요. 다음 문장. 다음 문장."
# inputs = tokenizer(text, return_tensors="pt")
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
print(inputs.keys())
print(inputs.token_type_ids)
# print(inputs2)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
# 모델을 통해 숨겨진 상태와 주의력 얻기
with torch.no_grad():
    outputs = model(**inputs)
    hidden_states = outputs.hidden_states
    attentions = outputs.attentions

# 필요한 벡터들 추출
last_cls_vector = hidden_states[-1][:, 0, :]  # 마지막 레이어의 CLS 위치
fourth_last_cls_vector = hidden_states[-4][:, 0, :]  # 마지막에서 4번째 레이어의 CLS 위치

# Attention Mask 적용 및 평균 풀링
attention_mask = inputs['attention_mask'].unsqueeze(-1).expand_as(hidden_states[-1])
print("attention_mask: ", attention_mask.shape)

masked_last_layer = hidden_states[-1][:, 1:, :] * attention_mask[:, 1:, :]
print("masked_last_layer: ", masked_last_layer.shape)

mean_pooled_vector = masked_last_layer.mean(dim=1)
print("mean_pooled_vector", mean_pooled_vector.shape)

# 벡터들 연결
concatenated_vector = torch.cat((last_cls_vector, fourth_last_cls_vector, mean_pooled_vector), dim=0)

# 결과 확인
print(concatenated_vector.shape)  # 결과 벡터의 크기 확인


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


attention_mask:  torch.Size([1, 18, 768])
masked_last_layer:  torch.Size([1, 17, 768])
mean_pooled_vector torch.Size([1, 768])
torch.Size([3, 768])


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput


class BertConcatClassifier(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BertConcatClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name, output_hidden_states=True)
        self.num_labels = num_labels
        #classification part
        self.conv = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=1)  # 3x768 -> 1x768
        self.relu = nn.ReLU()
        self.linear = nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states

        last_cls_vector = hidden_states[-1][:, 0, :]
        fourth_last_cls_vector = hidden_states[-4][:, 0, :]

        attention_mask_expanded = attention_mask.unsqueeze(-1).expand_as(hidden_states[-1])
        masked_last_layer = hidden_states[-1][:, 1:, :] * attention_mask_expanded[:, 1:, :]
        mean_pooled_vector = masked_last_layer.mean(dim=1)

        concatenated_vector = torch.cat((last_cls_vector.unsqueeze(1), fourth_last_cls_vector.unsqueeze(1), mean_pooled_vector.unsqueeze(1)), dim=1)
        # print("concatenated_vector shape: ", concatenated_vector.shape)


        conv_output = self.conv(concatenated_vector).squeeze(2)
        conv_output = self.relu(conv_output)
        # print("conv_output shape: ", conv_output.shape)

        logits = self.linear(conv_output)
        logits = logits.squeeze(1)
        # print("logits shape: ", logits.shape)

        # 손실 계산
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


# 모델 초기화 및 사용 예
model_name = "klue/bert-base"
num_labels = 60  # 예시: 이진 분류
model = BertConcatClassifier(model_name, num_labels)

# 입력 데이터 준비
tokenizer = AutoTokenizer.from_pretrained(model_name)
texts = ["이 예시 문장은 BERT 모델 입력으로 사용됩니다.", "두번째 문장입니다"]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# 모델 실행
outputs = model(inputs['input_ids'], inputs['attention_mask'])
print(outputs.logits)


tensor([[-0.2491,  0.1588, -0.0267, -0.0480, -0.0964, -0.2583,  0.0462,  0.4520,
         -0.0763, -0.1418,  0.1873,  0.1385,  0.1164, -0.0502, -0.0376, -0.0686,
          0.0679,  0.0523, -0.1752,  0.2612, -0.3228, -0.0419, -0.1934,  0.0266,
         -0.0169, -0.2789,  0.1039,  0.0569, -0.0485,  0.0940, -0.0268, -0.1753,
          0.1975, -0.2476,  0.0033, -0.1789, -0.2562, -0.0534,  0.1558,  0.0839,
          0.0427,  0.2631, -0.1021, -0.1648,  0.0771,  0.2237,  0.1687,  0.0843,
         -0.0166, -0.2250, -0.2079,  0.0204,  0.1897, -0.1111, -0.2029,  0.0028,
         -0.1488,  0.0170,  0.0249,  0.1451],
        [-0.3895,  0.1245,  0.2137, -0.1905,  0.0563, -0.0469,  0.0540,  0.2275,
         -0.0295, -0.1305,  0.1658,  0.2042,  0.2697, -0.0085,  0.1053, -0.1104,
          0.1778, -0.0224, -0.0673,  0.0442, -0.4062, -0.0140, -0.0935,  0.0182,
          0.0771, -0.1732,  0.0053, -0.0404,  0.0130,  0.1624, -0.0751, -0.1606,
          0.1358, -0.3286, -0.3066, -0.2332, -0.0670,  0.1843, 

# Training part

In [ ]:
from transformers import EarlyStoppingCallback
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# 모델 초기화 및 사용 예
model_name = "klue/bert-base"
num_labels = 60  # 예시: 이진 분류
model = BertConcatClassifier(model_name, num_labels).to(device)

def compute_metrics(pred):
    # 예측값에서 가장 큰 로짓의 인덱스를 예측 클래스로 사용
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    # average='macro'를 사용하여 각 클래스에 동일한 가중치를 부여
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/struc_ver3/results',
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=2000,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/truc_ver3/logs',
    logging_steps=10,
    save_steps=800,
    eval_steps=400,
    eval_strategy="steps",
    save_total_limit=None,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    gradient_accumulation_steps=2  # Use gradient accumulation
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# checkpoint_path = "/content/drive/MyDrive/struc_ver3/results/checkpoint-19600"

# Start training
trainer.train()



# Save the best model
model_path = training_args.output_dir + '/best_model'
model.save_pretrained(model_path)

Step,Training Loss,Validation Loss
400,4.020100,4.013752
800,3.772600,3.661162
1200,3.519800,3.348335
1600,3.378500,3.176636
2000,3.318900,3.116077
2400,3.178000,3.075073
2800,3.170800,3.021137
3200,3.166000,3.007940
3600,3.172800,2.973966
4000,3.154500,2.933302


AttributeError: 'BertConcatClassifier' object has no attribute 'save_pretrained'